I created this notebook as part of the Capstone project which I have to obtain my certification "IBM Data science" from Coursera.

In [626]:
import pandas as pd

In [627]:
import numpy as np

In [628]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Let's scrap !
<p>First, we are importing the libraries necessary to assign the link and read the source code</p>
<p>After reading the source code, we'll find the class and the balises</p>

In [629]:
# import library
import requests
from bs4 import BeautifulSoup

# assign the link
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

# read the source code
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify()[9270:10000])

      <table class="wikitable sortable">
       <tbody>
        <tr>
         <th>
          Postcode
         </th>
         <th>
          Borough
         </th>
         <th>
          Neighborhood
         </th>
        </tr>
        <tr>
         <td>
          M1A
         </td>
         <td>
          Not assigned
         </td>
         <td>
          Not assigned
         </td>
        </tr>
        <tr>
         <td>
          M2A
         </td>
         <td>
          Not assigned
         </td>
         <td>
          Not assigned
         </td>
        </tr>
        <tr>
         <td>
          M3A
         </td>
         <td>
          <a href="/wiki/North_York" title="North York">
           North York
   


In [630]:
# find class 'wikitable sortable' in the HTML script
my_table = soup.find('table',{'class' : 'wikitable sortable'})
my_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_

In [631]:
# find 'th' to know columns name
col_links = my_table.find_all('th')
col_links

[<th>Postcode</th>, <th>Borough</th>, <th>Neighborhood
 </th>]

In [632]:
# find 'td' to recover data
data_links = my_table.find_all('td')

# Check
data_links[:9]

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>]

## Let's tranforsm !
<p> We are going to convert the previous data to list, array and finally dataframe. </p>
<p> First, we are going to create a list to have the columns names. </p>
<p> Then, we 'll create an array to organise data. </p>
<p> Finally, we'll concatenate list and array to a dataframe. </p>

In [633]:
# transform to a list
col = []
for link in col_links:
    col.append(link.get_text())
print(col)

['Postcode', 'Borough', 'Neighborhood\n']


In [634]:
# tranform to a list
data = []

for link in data_links:
    l = link.get_text()
    data.append(l)

# transform to an array
n_col = 3
n_row = int(len(data) / n_col)
data_array = np.array(data).reshape((n_row, n_col))

In [635]:
# finally create a df from columns name and data
data_df = pd.DataFrame(data_array, columns=col)
data_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [636]:
# clean columns name
data_df.columns = [data_df.columns[0], data_df.columns[1], data_df.columns[2].replace('\n', '')]
data_df.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

In [637]:
# clean data
data_df['Neighborhood'] = data_df['Neighborhood'].str.replace('\n', '')
data_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Let's clean !
<p> We are going to clean the dataframe </p>
<p> For that, we 'll do :
    <ul>
      <li>Copy the original dataframe</li>
      <li>Filter</li>
      <li>Summarize neighbourhood data</li>
    </ul> </p>
<p> What a program ! </p>

In [638]:
# copy of original df
data_df_clean = data_df.copy()

# Ignore cells with a borough that is Not assigned
data_df_clean = data_df_clean[data_df_clean['Borough'] != 'Not assigned']

In [639]:
# groupby to move Neighbourhood values from row to column
data_df_clean = data_df_clean.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: pd.Series(list(x))).unstack().reset_index()

# Rename columns name
data_df_clean.columns = ['Postcode'
                         , 'Borough'
                         , 'Neighborhood1'
                         , 'Neighborhood2'
                         , 'Neighborhood3'
                         , 'Neighborhood4'
                         , 'Neighborhood5'
                         , 'Neighborhood6'
                         , 'Neighborhood7'
                         , 'Neighborhood8']

# check the transformation
data_df_clean.head()

,Postcode,Borough,Neighborhood1,Neighborhood2,Neighborhood3,Neighborhood4,Neighborhood5,Neighborhood6,Neighborhood7,Neighborhood8
0,M1B,Scarborough,Rouge,Malvern,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [640]:
# create an empty column
data_df_clean['Neighborhood'] = ''

# Create Neighbourhood list
nbh_list = ['Neighborhood1'
            , 'Neighborhood2'
            , 'Neighborhood3'
            , 'Neighborhood4'
            , 'Neighborhood5'
            , 'Neighborhood6'
            , 'Neighborhood7'
            , 'Neighborhood8']

# concatenate Neighbourhood values to new column
for i, n in enumerate(nbh_list):
    if i == 0:
        data_df_clean['Neighborhood'] = data_df_clean[n].fillna('')
    else:
        data_df_clean['Neighborhood'] = data_df_clean['Neighborhood'] + ', ' + data_df_clean[n].fillna('')       


# Drop useless columns
data_df_clean.drop(nbh_list, axis=1, inplace=True)

# Replace useless characters with regex
data_df_clean.replace(to_replace=r'[,+\s]{3,}', value='', regex=True, inplace=True)

# Check modifications
data_df_clean.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [641]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
data_df_clean['Neighborhood'] = np.where(data_df_clean['Neighborhood'] == 'Not assigned'
                                          , data_df_clean['Borough']
                                          , data_df_clean['Neighborhood'])

# Check the transformation with an example
data_df_clean[data_df_clean['Postcode'] == 'M7A']

,Postcode,Borough,Neighborhood
85,M7A,Downtown Toronto,Queen's Park


In [642]:
# use the .shape method to print the number of rows
print("The dataframe has", data_df_clean.shape[0], 'rows.')

The dataframe has 103 rows.


## Let's get the latitude and the longitude coordinates

<p>Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.</p>

<p>For that, we will use the csv file 'Geospatial_Coordinates.csv' to create a new dataframe.</p>

In [643]:
#data_df_clean = data_df_clean[data_df_clean['Borough'].str.contains('Toronto')]

In [644]:
# create df from 'Geospatial_Coordinates.csv'
coordinates_df = pd.read_csv('datasets/Geospatial_Coordinates.csv')

# merge main df with coordinates_df
data_df_clean = pd.merge(data_df_clean, coordinates_df, left_on = 'Postcode', right_on = 'Postal Code')
del data_df_clean['Postal Code']

# check the result
data_df_clean.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [645]:
data_df_clean.shape

(103, 5)

In [646]:
data_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode        103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


## Define Foursquare Credentials and Version

In [647]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

CLIENT_ID = 'UI4D4LFL50K4NJ1MDLZPNVYRSK1K1G11IWGIK5WEHJOPH4HA' # your Foursquare ID
CLIENT_SECRET = 'GB3FGOCPYKD1E5U5HEUEDVK5WUGBYJ2VB24GYW4UWTO4QTP5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UI4D4LFL50K4NJ1MDLZPNVYRSK1K1G11IWGIK5WEHJOPH4HA
CLIENT_SECRET:GB3FGOCPYKD1E5U5HEUEDVK5WUGBYJ2VB24GYW4UWTO4QTP5


In [648]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [649]:
toronto_venues = getNearbyVenues(names=data_df_clean['Neighborhood'],
                                   latitudes=data_df_clean['Latitude'],
                                   longitudes=data_df_clean['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [650]:
print(toronto_venues.shape)
#toronto_venues.head()
toronto_venues

(2226, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


In [651]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,56,56,56,56,56,56


In [652]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


## Analyze Each Neighborhood

In [653]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [654]:
toronto_onehot.shape

(2226, 274)

In [655]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
1,Agincourt,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.017857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [656]:
toronto_grouped.shape

(100, 274)

In [657]:
toronto_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 274 entries, Neighborhood to Women's Store
dtypes: float64(273), object(1)
memory usage: 214.1+ KB


In [658]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+ hood +"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                       venue  freq
0               Skating Rink   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground  0.25
1                 Park  0.25
2          Coffee Shop  0.25
3  Arts & Crafts Store  0.25
4   Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.22
1           Pizza Place  0.11
2           Video Store  0.11
3        Sandwich Place  0.11
4  Fast Food Restaurant  0.11


----Alderwood, Long Branch----
       

                  venue  freq
0        Discount Store   0.2
1  Gym / Fitness Center   0.2
2                   Gym   0.2
3         Grocery Store   0.2
4    Athletics & Sports   0.2


----Downsview West----
           venue  freq
0  Grocery Store  0.33
1           Bank  0.17
2  Shopping Mall  0.17
3           Park  0.17
4          Hotel  0.17


----Downsview, North Park, Upwood Park----
                        venue  freq
0  Construction & Landscaping   0.2
1                        Park   0.2
2            Basketball Court   0.2
3               Deli / Bodega   0.2
4                      Bakery   0.2


----East Birchmount Park, Ionview, Kennedy Park----
                venue  freq
0      Discount Store  0.29
1    Department Store  0.14
2          Hobby Shop  0.14
3         Bus Station  0.14
4  Chinese Restaurant  0.14


----East Toronto----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2   Convenience Store  0.33
3         Yoga Studio  0.00
4  Mexican 

                        venue  freq
0                        Park  0.50
1                       Trail  0.25
2                  Playground  0.25
3                 Yoga Studio  0.00
4  Modern European Restaurant  0.00


----Roselawn----
               venue  freq
0             Garden   0.5
1       Home Service   0.5
2   Malay Restaurant   0.0
3             Market   0.0
4  Martial Arts Dojo   0.0


----Rouge, Malvern----
                        venue  freq
0        Fast Food Restaurant   0.5
1                  Print Shop   0.5
2  Modern European Restaurant   0.0
3           Mobile Phone Shop   0.0
4          Miscellaneous Shop   0.0


----Runnymede, Swansea----
                venue  freq
0                Café  0.09
1         Coffee Shop  0.06
2  Italian Restaurant  0.06
3    Sushi Restaurant  0.06
4         Pizza Place  0.06


----Ryerson, Garden District----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.06
2                       C

In [659]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [660]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Restaurant,Vegetarian / Vegan Restaurant
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Arts & Crafts Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Video Store,Sandwich Place,Fast Food Restaurant,Beer Store,Pizza Place,Pharmacy,Eastern European Restaurant,Dumpling Restaurant
4,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Coffee Shop,Skating Rink,Pharmacy,Sandwich Place,Pub,Pool,Discount Store


In [661]:
neighborhoods_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
Neighborhood              100 non-null object
1st Most Common Venue     100 non-null object
2nd Most Common Venue     100 non-null object
3rd Most Common Venue     100 non-null object
4th Most Common Venue     100 non-null object
5th Most Common Venue     100 non-null object
6th Most Common Venue     100 non-null object
7th Most Common Venue     100 non-null object
8th Most Common Venue     100 non-null object
9th Most Common Venue     100 non-null object
10th Most Common Venue    100 non-null object
dtypes: object(11)
memory usage: 8.7+ KB


## Cluster Neighborhoods

In [662]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]
set(kmeans.labels_)

{0, 1, 2, 3, 4}

In [663]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data_df_clean

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 16 columns):
Postcode                  103 non-null object
Borough                   103 non-null object
Neighborhood              103 non-null object
Latitude                  103 non-null float64
Longitude                 103 non-null float64
Cluster Labels            101 non-null float64
1st Most Common Venue     101 non-null object
2nd Most Common Venue     101 non-null object
3rd Most Common Venue     101 non-null object
4th Most Common Venue     101 non-null object
5th Most Common Venue     101 non-null object
6th Most Common Venue     101 non-null object
7th Most Common Venue     101 non-null object
8th Most Common Venue     101 non-null object
9th Most Common Venue     101 non-null object
10th Most Common Venue    101 non-null object
dtypes: float64(3), object(13)
memory usage: 13.7+ KB


In [664]:
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notnull()]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [665]:
toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 102
Data columns (total 16 columns):
Postcode                  101 non-null object
Borough                   101 non-null object
Neighborhood              101 non-null object
Latitude                  101 non-null float64
Longitude                 101 non-null float64
Cluster Labels            101 non-null int32
1st Most Common Venue     101 non-null object
2nd Most Common Venue     101 non-null object
3rd Most Common Venue     101 non-null object
4th Most Common Venue     101 non-null object
5th Most Common Venue     101 non-null object
6th Most Common Venue     101 non-null object
7th Most Common Venue     101 non-null object
8th Most Common Venue     101 non-null object
9th Most Common Venue     101 non-null object
10th Most Common Venue    101 non-null object
dtypes: float64(2), int32(1), object(13)
memory usage: 13.0+ KB


## Let's map !

In [666]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = 43.7001100
longitude = -79.4163000

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [667]:
for i in range(kclusters):
    display(toronto_merged[toronto_merged['Cluster Labels'] == i])
    print("----------------------------------------------------")

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Intersection,Women's Store,Doner Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Dog Run,Dim Sum Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Smoke Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Discount Store,Hobby Shop,Coffee Shop,Bus Station,Chinese Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Soccer Field,Park,Fast Food Restaurant,Metro Station,Intersection,Donut Shop,Dog Run,Doner Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Motel,Intersection,American Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,College Stadium,General Entertainment,Café,Skating Rink,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,0,Indian Restaurant,Pet Store,Furniture / Home Store,Vietnamese Restaurant,Chinese Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run


----------------------------------------------------


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M2L,North York,"Silver Hills, York Mills",43.75749,-79.374714,1,Cafeteria,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dessert Shop


----------------------------------------------------


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,2,Park,Coffee Shop,Playground,Arts & Crafts Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
23,M2P,North York,York Mills West,43.752758,-79.400049,2,Park,Bank,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,2,Park,Airport,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,M4J,East York,East Toronto,43.685347,-79.338106,2,Convenience Store,Park,Coffee Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Park,Restaurant,Tennis Court,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,2,Park,Trail,Jewelry Store,Sushi Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,2,Park,Women's Store,Market,Fast Food Restaurant,Comfort Food Restaurant,Comic Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant


----------------------------------------------------


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,3,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242,3,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


----------------------------------------------------


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Golf Course,Bar,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,4,Golf Course,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


----------------------------------------------------
